<a href="https://colab.research.google.com/github/gopalaniyengar/ITSP_BigBrainTeam_CNN/blob/Gopalan/ITSP_Proj_MK2_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import sklearn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms

In [4]:
transform=transforms.ToTensor()
trainset=datasets.EMNIST(root='/content/drive/My Drive/Colab Notebooks/data', split= 'byclass', train=True, download=True, transform=transform)
testset=datasets.EMNIST(root='/content/drive/My Drive/Colab Notebooks/data', split= 'byclass', train=False, download=True, transform=transform)
print(len(trainset))
print(len(testset))
print(len(trainset)+len(testset))
print(1)  

697932
116323
814255
1


In [5]:
batch=32
numberofbatches=697932/32
print(numberofbatches)
numberofbatches=21811
print(numberofbatches)
trainloadeddata=DataLoader(trainset, batch_size=batch, shuffle= True)
testloadeddata=DataLoader(testset, batch_size=batch, shuffle= True)
#for batchno,(image, labels) in enumerate(trainloadeddata):
    #if  batchno==1:
     #print(image, labels)

21810.375
21811


In [6]:
class ModelOfNet(nn.Module):
    def __init__(self):
        super(ModelOfNet,self).__init__()

        self.convolve=nn.Sequential(
                nn.Conv2d(1,8,3,1,1),
                nn.ReLU(),
                nn.Conv2d(8,16,3,1,1),
                nn.ReLU(),
                nn.BatchNorm2d(16),
                nn.MaxPool2d(2,2),
                nn.Conv2d(16,32,3,1,1),
                nn.ReLU(),
                nn.BatchNorm2d(32),
                nn.MaxPool2d(2,2),
        )
        self.fullyconnect=nn.Linear(1568,62)

    def forward(self, x):
        x=self.convolve(x)
        x=x.view(x.size(0),-1)
        x=self.fullyconnect(x)
        return x 
    print('I sure hope this model works fine this time without any errors')

I sure hope this model works fine this time without any errors


In [7]:
MYNN=ModelOfNet()
print(MYNN)
classwts=torch.load('/content/drive/My Drive/Colab Notebooks/lossweights.pt')
print(classwts)
print(classwts.dtype)
lossfunction=torch.nn.CrossEntropyLoss(weight=classwts)
updatefunction=torch.optim.Adam(MYNN.parameters(), lr=0.00001)
print('completed running block')

ModelOfNet(
  (convolve): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fullyconnect): Linear(in_features=1568, out_features=62, bias=True)
)
tensor([0.3255, 0.2933, 0.3291, 0.3203, 0.3357, 0.3583, 0.3288, 0.3148, 0.3316,
        0.3326, 1.7570, 2.9028, 1.1152, 2.4676, 2.2815, 1.2260, 4.4724, 3.5714,
        0.9423, 2.9923, 4.5612, 2.2177, 1.2505, 1.3666, 0.4506, 1.3486, 4.3213,
        2.2190, 0.5421, 1.146

In [0]:
def trainingtime(epochs):
  avglossoverepochs=[]
  accuracyoverepochs=[]
  for i in range(epochs):
    sum=0.0
    avgaccuracyoverepoch=0.0
    sumbatch=0.0
    for batchno,(image, labels) in enumerate(trainloadeddata):
      updatefunction.zero_grad()
      resultofmodel=MYNN(image)
      losscalc=lossfunction(resultofmodel, labels)
      resultofmodel=F.softmax(resultofmodel, dim=1)
      #print(resultofmodel.shape)
      pred=torch.argmax(resultofmodel, dim=1)
      #print(pred)
      count=0.0
      assert pred.shape[0]==labels.shape[0]
      for j in range(pred.shape[0]):
        if pred[j]==labels[j]:
          count=count+1.0
      batchaccuracy=(count/pred.shape[0])*100
      sum=sum+losscalc
      sumbatch=sumbatch+losscalc
      losscalc.backward()
      updatefunction.step()
      if batchno%100==True:
        print(i, batchno, "Batch Loss:", losscalc,"Batch Accuracy=", batchaccuracy,"%", "Average Loss Over 100 Batches:", (sumbatch/100))
        sumbatch=0.0
      avgaccuracyoverepoch=avgaccuracyoverepoch+(batchaccuracy/numberofbatches)
    print("Average Loss over Epoch:", (sum/numberofbatches))
    print("Average Accuracy over Epoch:", avgaccuracyoverepoch, "%")
    accuracyoverepochs.append(avgaccuracyoverepoch)
    avglossoverepochs.append((sum/numberofbatches))
  print('hopefully i reach this part of the function')
  print(accuracyoverepochs)
  print(avglossoverepochs)
  

In [9]:
t=torch.tensor([1,2,3,4,5,6,7,8,9], dtype=torch.float32)
t=F.softmax(t, dim=-1)
print(t)
print(torch.max(t).item())
k=np.array(t)
print(k)
ind=np.argmax(k)
print(ind)

tensor([2.1208e-04, 5.7649e-04, 1.5671e-03, 4.2597e-03, 1.1579e-02, 3.1475e-02,
        8.5559e-02, 2.3257e-01, 6.3220e-01])
0.6321986317634583
[2.1207902e-04 5.7649054e-04 1.5670639e-03 4.2597209e-03 1.1579122e-02
 3.1475317e-02 8.5558780e-02 2.3257288e-01 6.3219863e-01]
8


In [15]:
numberofepochs=7
trainingtime(numberofepochs)

0 1 Batch Loss: tensor(0.7334, grad_fn=<NllLossBackward>) Batch Accuracy= 65.625 % Average Loss Over 100 Batches: tensor(0.0166, grad_fn=<DivBackward0>)
0 101 Batch Loss: tensor(1.1657, grad_fn=<NllLossBackward>) Batch Accuracy= 62.5 % Average Loss Over 100 Batches: tensor(0.9033, grad_fn=<DivBackward0>)
0 201 Batch Loss: tensor(0.6200, grad_fn=<NllLossBackward>) Batch Accuracy= 87.5 % Average Loss Over 100 Batches: tensor(0.8644, grad_fn=<DivBackward0>)
0 301 Batch Loss: tensor(0.6399, grad_fn=<NllLossBackward>) Batch Accuracy= 87.5 % Average Loss Over 100 Batches: tensor(0.8737, grad_fn=<DivBackward0>)
0 401 Batch Loss: tensor(0.5496, grad_fn=<NllLossBackward>) Batch Accuracy= 81.25 % Average Loss Over 100 Batches: tensor(0.8670, grad_fn=<DivBackward0>)
0 501 Batch Loss: tensor(1.1890, grad_fn=<NllLossBackward>) Batch Accuracy= 71.875 % Average Loss Over 100 Batches: tensor(0.8557, grad_fn=<DivBackward0>)
0 601 Batch Loss: tensor(0.6264, grad_fn=<NllLossBackward>) Batch Accuracy= 78.